# Website Generator for Lightroom Exports

This Jupyter Notebook automatically generates HTML gallery pages from Adobe Lightroom Web Exports. 

It ensures a consistent structure and visual style across multiple pages while giving you full editorial control over layout, navigation, quotes, and page content.

## What It Does

- Parses exported Lightroom HTML files to extract image metadata (file name, dimensions, title).
- Organizes images into responsive Bootstrap rows:
  - **Portrait** images are arranged in rows of 3.
  - **Landscape** images are arranged in rows of 2.
- Builds fully styled HTML pages using Bootstrap components like navbar, jumbotron, grid layout, and footer.
- Includes a static `index.html` homepage with a manually defined welcome section and image.
- Ensures all pages share a uniform layout and dynamic navbar.


## Structure of This Notebook

### 1. Page Configuration (`pages`)
A list of dictionaries, each representing a page to generate. Each dictionary includes:
- Lightroom export folder
- Output HTML file name
- Page title and intro text
- Quote and author
- Active navigation link (to highlight in the navbar)

### 2. Shared Navbar Definition
The `navbar_links` are automatically extracted from the `pages` list so every page has consistent navigation.

### 3. HTML Generation Functions
Reusable functions generate each section of the page:
- `generate_header()`
- `generate_navbar(nav_links, active_page)`
- `generate_jumbotron(title, heading, paragraph)`
- `generate_image_section(images, folder)`
- `generate_quote_section(quote, author)`
- `generate_footer()`

### 4. `extract_images_from_lightroom()`
Parses the `index.html` file from each Lightroom export to extract image metadata including width and height (used to determine orientation).

### 5. `generate_full_html()`
Assembles a complete HTML page using the modular functions and a `page` dictionary.

### 6. `generate_index_html()`
Generates the static homepage using fixed content, but still using the shared layout and navbar.

### 7. `main()`
Loops through all configured pages, extracts images, generates HTML, and writes each page to disk. Also builds the `index.html` using its own content and the same layout.

---

## How to Add a New Page

Just add a new dictionary to the `pages` list, like this:

```python
{
    "lightroom_folder": "LR_Export_NewGallery",
    "output_file": "new-gallery.html",
    "page_title": "New Gallery",
    "active_nav": "new-gallery.html",
    "intro_heading": "A new perspective.",
    "intro_paragraph": "Photos captured in a recent journey.",
    "quote": "The camera is an instrument that teaches people how to see without a camera.",
    "quote_author": "Dorothea Lange"
}

# Important Instruction: Export the Track Gallery - the others do not put the Hight/Width info in the HTML

In [121]:
# Page-specific settings (set these per page)

In [122]:
pages = [
    {
        "lightroom_folder": "LR_Export_Moments",
        "output_file": "moments.html",
        "page_title": "Moments",
        "active_nav": "moments.html",
        "intro_heading": "",
        "intro_paragraph": "Street photography is not about what we see, but about what reveals itself when we slow down.",
        "quote": "To me, photography is an art of observation.<br>It’s about finding something interesting in an ordinary place.",
        "quote_author": "Elliott Erwitt"
    },
    {
        "lightroom_folder": "LR_Export_Portraits",
        "output_file": "portraits.html",
        "page_title": "Portraits",
        "active_nav": "portraits.html",
        "intro_heading": "",
        "intro_paragraph": "Faces don’t ask to be explained.<br>They invite us into an unspoken conversation—a gaze that doesn’t give answers, but shifts something in the one who looks.",
        "quote": "A portrait is not made in the camera but on either side of it.",
        "quote_author": "Edward Steichen"
    }
    # Add more pages here...
]

Build the navigation based on the data structure above

In [124]:
def generate_navbar(nav_links, active_page):
    """Generate the navbar using provided links and active page."""
    nav_items = []
    for href, label in nav_links:
        active_class = " active" if href == active_page else ""
        sr_only = ' <span class="sr-only">(current)</span>' if href == active_page else ""
        nav_items.append(
            f'<li class="nav-item{active_class}"><a class="nav-link" href="{href}">{label}{sr_only}</a></li>'
        )

    return f"""
    <nav class="navbar navbar-expand-lg navbar-dark bg-dark">
        <a class="navbar-brand" href="#">Pascal Reusch Photography</a>
        <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarSupportedContent"
                aria-controls="navbarSupportedContent" aria-expanded="false" aria-label="Toggle navigation">
            <span class="navbar-toggler-icon"></span>
        </button>
        <div class="collapse navbar-collapse" id="navbarSupportedContent">
            <ul class="navbar-nav mr-auto">
                {''.join(nav_items)}
            </ul>
        </div>
    </nav>
    """

Generates a header that is the same on every page.

In [126]:
def generate_header():
    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <title>Pascal Reusch Photography</title>
        <link href="css/bootstrap-4.4.1.css" rel="stylesheet">
        <link href="css/lightbox.min.css" rel="stylesheet">
    </head>
    <body>
    """

The generate_index_html() function uses static content and a dynamic navbar.

In [128]:
def generate_index_html(navbar_links):
    return (
        generate_header()
        + generate_navbar(navbar_links, active_page="index.html")
        + """
        <section>
          <div class="jumbotron text-center mt-2 py-2">
            <div class="container">
              <div class="row">
                <div class="col-12">
                  <h1>Pascal Reusch</h1>
                  <h5>Learn to see the beauty in all things!</h5>
                </div>
              </div>
            </div>
          </div>
        </section>
        <section>
          <div class="container">
            <div class="row">
              <div class="col-md-12 col-12 text-center col-xl-12">
                <img class="img-fluid" src="images/DSC05209.jpg" alt="">
                <h5>Glacier National Parc (Montana, USA)</h5>
              </div>
            </div>
          </div>
        </section>
        <hr>
        <section>
          <div class="container text-center">
            <h5>To connect and exchange art and ideas send an email:</h5>
            <h5><a href="mailto:reuschphotography@gmail.com">reuschphotography@gmail.com</a></h5>
          </div>
        </section>
        <hr>
        """
        + generate_footer()
    )

In [129]:
import re

def extract_images_from_lightroom(file_path):
    """Extract image data from Lightroom HTML."""
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()

    # Locate the LR.images block
    start_index = content.find("LR.images = [")
    if start_index == -1:
        raise ValueError("Could not find 'LR.images = [' in the file.")

    start_index += len("LR.images = [")  # Move past 'LR.images = ['
    end_index = content.find("]", start_index)
    if end_index == -1:
        raise ValueError("Could not find the closing ']' for LR.images.")

    # Extract the block as raw text
    raw_data = content[start_index:end_index + 1]

    # Split the raw data into lines or individual objects
    raw_items = re.split(r"},\s*{", raw_data.strip()[1:-1])  # Split by objects
    images = []

    for item in raw_items:
        # Ensure each item is properly wrapped with braces
        item = "{" + item.strip() + "}"
        item = item.replace("&quot;", '"')  # Replace HTML entities with quotes

        # Extract key-value pairs manually
        fields = re.findall(r'"(\w+)":\s*"([^"]*?)"', item)
        image_data = {}

        for key, value in fields:
            # Parse numeric fields as integers, leave others as strings
            if key in {"largeWidth", "largeHeight"}:
                try:
                    image_data[key] = int(value)
                except ValueError:
                    image_data[key] = 0  # Fallback for invalid integers
            else:
                image_data[key] = value.strip()

        # Ensure required keys exist
        if "largeWidth" in image_data and "largeHeight" in image_data:
            images.append(image_data)
        else:
            print(f"Skipping image due to missing dimensions: {item}")

    return images

In [130]:
def generate_jumbotron(title, heading, paragraph):
    return f"""
    <section>
        <div class="jumbotron text-center mt-2 py-2">
            <div class="container">
                <div class="row">
                    <div class="col-12">
                        <h1>{title}</h1>
                        <h4>{heading}</h4>
                        <p>{paragraph}</p>
                    </div>
                </div>
            </div>
        </div>
    </section>
    """


def generate_image_section(images, lightroom_folder):
    """Generate the image gallery section."""
    html_content = '<section><div class="container">'
    portrait_row = []
    landscape_row = []

    for image in images:
        # Determine orientation of the image
        orientation = "portrait" if int(image["largeHeight"]) > int(image["largeWidth"]) else "landscape"

        # Create the basic image item without column definitions
        html_item = f"""
            <a href="{lightroom_folder}/images/large/{image['exportFilename']}.jpg" data-lightbox="gallery" data-title="{image['title']}">
                <img class="img-fluid" src="{lightroom_folder}/images/large/{image['exportFilename']}.jpg" alt="{image['title']}">
            </a>
            <h5>{image['title']}</h5>
        """

        if orientation == "portrait":
            portrait_row.append(html_item)
            if len(portrait_row) == 3:  # Once 3 portraits are collected, add the row
                html_content += '<div class="row mt-3">' + ''.join(
                    f'<div class="col-md-4 col-12 text-center">{img}</div>' for img in portrait_row
                ) + '</div>'
                portrait_row = []
        else:  # Landscape
            landscape_row.append(html_item)
            if len(landscape_row) == 2:  # Once 2 landscapes are collected, add the row
                html_content += '<div class="row mt-3">' + ''.join(
                    f'<div class="col-md-6 col-12 text-center">{img}</div>' for img in landscape_row
                ) + '</div>'
                landscape_row = []

    # Handle any remaining images
    if portrait_row:
        col_size = 12 // len(portrait_row)  # Dynamically adjust column size
        html_content += '<div class="row mt-3">' + ''.join(
            f'<div class="col-md-{col_size} col-12 text-center">{img}</div>' for img in portrait_row
        ) + '</div>'

    if landscape_row:
        col_size = 12 // len(landscape_row)  # Dynamically adjust column size
        html_content += '<div class="row mt-3">' + ''.join(
            f'<div class="col-md-{col_size} col-12 text-center">{img}</div>' for img in landscape_row
        ) + '</div>'

    html_content += '</div></section>'
    return html_content

def generate_quote(text, author):
    return f"""
    <section>
      <hr>
      <div class="container">
        <div class="row">
          <div class="col-12 text-center">
            <h4>{text}</h4>
            <h4>{author}</h4>
          </div>
        </div>
      </div>
    </section>
    """

def generate_footer():
    """Generate the footer."""
    return """
    <footer class="text-center">
        <div class="container">
            <div class="row">
                <div class="col-12">
                    <p>Copyright © Pascal Reusch Photography. All rights reserved.</p>
                </div>
            </div>
        </div>
    </footer>
    <!-- Include Bootstrap JavaScript -->
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script src="js/popper.min.js"></script>
    <script src="js/bootstrap-4.4.1.js"></script>
    <script src="js/lightbox.min.js"></script>
    </body>
    </html>
    """


def generate_full_html(images, page, navbar_links):
    return (
        generate_header()
        + generate_navbar(navbar_links, page["active_nav"])
        + generate_jumbotron(page["page_title"], page["intro_heading"], page["intro_paragraph"])
        + generate_image_section(images, page["lightroom_folder"])
        + generate_quote(page["quote"], page["quote_author"])
        + generate_footer()
    )

In [131]:
def main():
    navbar_links = [("index.html", "Home")] + [(page["output_file"], page["page_title"]) for page in pages]

    # Generate static index page
    with open("index.html", "w", encoding="utf-8") as f:
        f.write(generate_index_html(navbar_links))
    print("✓ index.html created")

    # Generate Lightroom-based gallery pages
    for page in pages:
        input_html = f"{page['lightroom_folder']}/index.html"
        images = extract_images_from_lightroom(input_html)
        html = generate_full_html(images, page, navbar_links)
        with open(page["output_file"], "w", encoding="utf-8") as f:
            f.write(html)
        print(f"✓ {page['output_file']} created")

if __name__ == "__main__":
    main()

✓ index.html created
✓ moments.html created
✓ portraits.html created
